In [74]:
import os
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split, ParameterGrid
from sklearn.feature_selection import RFE, SelectKBest, f_classif, VarianceThreshold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import GridSearchCV
import dotenv
import enum


dotenv.load_dotenv()

# ==== SETTINGS ====
IS_HYPERPARAMETER_TUNING = True
OUTLIER_THRESHOLD = 3.0
MODEL_MAIN_DIR_PATH = os.getenv("MODEL_MAIN_DIR")


featuresTypePathDict = {
    "bandpower": "/bandpower/",
    "coherence": "/coherence/",
    "hfd": "/hfd/",
    "relativepower": "/relativepower/",
    "cd": "/cd/", 
    "subbandpower": "/subbandpower/",
    "alphabeta": "/alphabeta/",
    "alphatheta": "/alphatheta/"
}

## Channels Index Mapping Enum

In [75]:
class ChannelsIndexMapping(enum.Enum):
    Fp1 = 0
    Fp2 = 1
    F3 = 2
    F4 = 3
    C3 = 4
    C4 = 5
    P3 = 6
    P4 = 7
    O1 = 8
    O2 = 9
    F7 = 10
    F8 = 11
    T7 = 12
    T8 = 13
    P7 = 14
    P8 = 15

## Interquatile Range (IQR) Method

Lower (e.g., 1.0 × IQR) → More values considered outliers (stricter).

Higher (e.g., 3.0 × IQR) → Fewer values considered outliers (looser, only extreme ones removed).

In [76]:
def remove_outliers_iqr_3d(data, threshold=OUTLIER_THRESHOLD):
    numEpochs, numBands, numChannels = data.shape
    mask = np.ones(numEpochs, dtype=bool)
    for band in range(numBands):
        for ch in range(numChannels):
            values = data[:, band, ch]
            Q1 = np.percentile(values, 25)
            Q3 = np.percentile(values, 75)
            IQR = Q3 - Q1
            lowerBound = Q1 - threshold * IQR
            upperBound = Q3 + threshold * IQR
            mask &= (values >= lowerBound) & (values <= upperBound)
    return data[mask], mask


def remove_outliers_iqr_2d(data, threshold=OUTLIER_THRESHOLD):
    numEpochs, numChannels = data.shape
    mask = np.ones(numEpochs, dtype=bool)
    for ch in range(numChannels):
        values = data[:, ch]
        Q1 = np.percentile(values, 25)
        Q3 = np.percentile(values, 75)
        IQR = Q3 - Q1
        lowerBound = Q1 - threshold * IQR
        upperBound = Q3 + threshold * IQR
        mask &= (values >= lowerBound) & (values <= upperBound)
    return data[mask], mask


def remove_outliers(data, threshold=OUTLIER_THRESHOLD):
    if len(data.shape) == 3:  
        return remove_outliers_iqr_3d(data, threshold)
    elif len(data.shape) == 2:  
        return remove_outliers_iqr_2d(data, threshold)
    else:
        raise ValueError("Data must be either 2D or 3D.")

## Prepare Features

In [77]:
def get_feature(featureTypes, modmaFeaturesDirPath, predctFeaturesDirPath, channelsSelectedSetLst, threshold=3):
    featuresPathDict = {}
    for featureType in featureTypes:
        featuresPathDict[f"hc{featureType.capitalize()}ModmaFeaturesPath"] = (
            modmaFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_HC.npy")
        featuresPathDict[f"hc{featureType.capitalize()}PredctFeaturesPath"] = (
            predctFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_HC.npy")
        featuresPathDict[f"mdd{featureType.capitalize()}ModmaFeaturesPath"] = (
            modmaFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_MDD.npy")
        featuresPathDict[f"mdd{featureType.capitalize()}PredctFeaturesPath"] = (
            predctFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_MDD.npy")
        featuresPathDict[f"hc{featureType.capitalize()}ModmaUnseenFeaturesPath"] = (
            modmaFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_HC_unseen.npy")
        featuresPathDict[f"hc{featureType.capitalize()}PredctUnseenFeaturesPath"] = (
            predctFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_HC_unseen.npy")
        featuresPathDict[f"mdd{featureType.capitalize()}ModmaUnseenFeaturesPath"] = (
            modmaFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_MDD_unseen.npy")
        featuresPathDict[f"mdd{featureType.capitalize()}PredctUnseenFeaturesPath"] = (
            predctFeaturesDirPath + featuresTypePathDict[featureType] + f"{featureType}_MDD_unseen.npy")

    hcFeaturesDict, mddFeaturesDict = {}, {}
    hcUnseenFeaturesDict, mddUnseenFeaturesDict = {}, {}

    for featureType in featureTypes:
        try:
            hcModmaFeatures = np.load(featuresPathDict[f"hc{featureType.capitalize()}ModmaFeaturesPath"])
            hcPredctFeatures = np.load(featuresPathDict[f"hc{featureType.capitalize()}PredctFeaturesPath"])
            mddModmaFeatures = np.load(featuresPathDict[f"mdd{featureType.capitalize()}ModmaFeaturesPath"])
            mddPredctFeatures = np.load(featuresPathDict[f"mdd{featureType.capitalize()}PredctFeaturesPath"])
            hcModmaUnseenFeatures = np.load(featuresPathDict[f"hc{featureType.capitalize()}ModmaUnseenFeaturesPath"])
            hcPredctUnseenFeatures = np.load(featuresPathDict[f"hc{featureType.capitalize()}PredctUnseenFeaturesPath"])
            mddModmaUnseenFeatures = np.load(featuresPathDict[f"mdd{featureType.capitalize()}ModmaUnseenFeaturesPath"])
            mddPredctUnseenFeatures = np.load(featuresPathDict[f"mdd{featureType.capitalize()}PredctUnseenFeaturesPath"])

            # Select channels based on channelsSelectedSetLst
            hcModmaFeatures = hcModmaFeatures[..., channelsSelectedSetLst]
            hcPredctFeatures = hcPredctFeatures[..., channelsSelectedSetLst]
            mddModmaFeatures = mddModmaFeatures[..., channelsSelectedSetLst]
            mddPredctFeatures = mddPredctFeatures[..., channelsSelectedSetLst]
            hcModmaUnseenFeatures = hcModmaUnseenFeatures[..., channelsSelectedSetLst]
            hcPredctUnseenFeatures = hcPredctUnseenFeatures[..., channelsSelectedSetLst]
            mddModmaUnseenFeatures = mddModmaUnseenFeatures[..., channelsSelectedSetLst]
            mddPredctUnseenFeatures = mddPredctUnseenFeatures[..., channelsSelectedSetLst]

            # Align dimensions before concatenation
            min_dim_hc = min(hcModmaFeatures.shape[1], hcPredctFeatures.shape[1])
            hcModmaFeatures = hcModmaFeatures[:, :min_dim_hc]
            hcPredctFeatures = hcPredctFeatures[:, :min_dim_hc]

            min_dim_mdd = min(mddModmaFeatures.shape[1], mddPredctFeatures.shape[1])
            mddModmaFeatures = mddModmaFeatures[:, :min_dim_mdd]
            mddPredctFeatures = mddPredctFeatures[:, :min_dim_mdd]

            min_dim_hc_unseen = min(hcModmaUnseenFeatures.shape[1], hcPredctUnseenFeatures.shape[1])
            hcModmaUnseenFeatures = hcModmaUnseenFeatures[:, :min_dim_hc_unseen]
            hcPredctUnseenFeatures = hcPredctUnseenFeatures[:, :min_dim_hc_unseen]

            min_dim_mdd_unseen = min(mddModmaUnseenFeatures.shape[1], mddPredctUnseenFeatures.shape[1])
            mddModmaUnseenFeatures = mddModmaUnseenFeatures[:, :min_dim_mdd_unseen]
            mddPredctUnseenFeatures = mddPredctUnseenFeatures[:, :min_dim_mdd_unseen]

            # Concatenate Modma and Predct features
            hcFeaturesDict[f"hc{featureType.capitalize()}Features"] = np.concatenate((hcModmaFeatures, hcPredctFeatures), axis=0)
            mddFeaturesDict[f"mdd{featureType.capitalize()}Features"] = np.concatenate((mddModmaFeatures, mddPredctFeatures), axis=0)
            hcUnseenFeaturesDict[f"hc{featureType.capitalize()}UnseenFeatures"] = np.concatenate((hcModmaUnseenFeatures, hcPredctUnseenFeatures), axis=0)
            mddUnseenFeaturesDict[f"mdd{featureType.capitalize()}UnseenFeatures"] = np.concatenate((mddModmaUnseenFeatures, mddPredctUnseenFeatures), axis=0)
        except FileNotFoundError as e:
            print(f"File not found: {e}")

    # Remove outliers
    for key in hcFeaturesDict.keys():
        hcFeaturesDict[key], _ = remove_outliers(hcFeaturesDict[key], threshold)
    for key in mddFeaturesDict.keys():
        mddFeaturesDict[key], _ = remove_outliers(mddFeaturesDict[key], threshold)
    for key in hcUnseenFeaturesDict.keys():
        hcUnseenFeaturesDict[key], _ = remove_outliers(hcUnseenFeaturesDict[key], threshold)
    for key in mddUnseenFeaturesDict.keys():
        mddUnseenFeaturesDict[key], _ = remove_outliers(mddUnseenFeaturesDict[key], threshold)

    # Find the minimum number of samples across all HC and MDD features
    min_hc_samples = min([hcFeaturesDict[key].shape[0] for key in hcFeaturesDict.keys()])
    min_mdd_samples = min([mddFeaturesDict[key].shape[0] for key in mddFeaturesDict.keys()])
    min_hc_unseen_samples = min([hcUnseenFeaturesDict[key].shape[0] for key in hcUnseenFeaturesDict.keys()])
    min_mdd_unseen_samples = min([mddUnseenFeaturesDict[key].shape[0] for key in mddUnseenFeaturesDict.keys()])

    # Truncate features to the minimum number of samples
    for key in hcFeaturesDict.keys():
        hcFeaturesDict[key] = hcFeaturesDict[key][:min_hc_samples]
    for key in mddFeaturesDict.keys():
        mddFeaturesDict[key] = mddFeaturesDict[key][:min_mdd_samples]
    for key in hcUnseenFeaturesDict.keys():
        hcUnseenFeaturesDict[key] = hcUnseenFeaturesDict[key][:min_hc_unseen_samples]
    for key in mddUnseenFeaturesDict.keys():
        mddUnseenFeaturesDict[key] = mddUnseenFeaturesDict[key][:min_mdd_unseen_samples]

    # Flatten features if 3D
    for key in hcFeaturesDict.keys():
        if len(hcFeaturesDict[key].shape) == 3:
            hcFeaturesDict[key] = hcFeaturesDict[key].reshape(hcFeaturesDict[key].shape[0], -1)
    for key in mddFeaturesDict.keys():
        if len(mddFeaturesDict[key].shape) == 3:
            mddFeaturesDict[key] = mddFeaturesDict[key].reshape(mddFeaturesDict[key].shape[0], -1)
    for key in hcUnseenFeaturesDict.keys():
        if len(hcUnseenFeaturesDict[key].shape) == 3:
            hcUnseenFeaturesDict[key] = hcUnseenFeaturesDict[key].reshape(hcUnseenFeaturesDict[key].shape[0], -1)
    for key in mddUnseenFeaturesDict.keys():
        if len(mddUnseenFeaturesDict[key].shape) == 3:
            mddUnseenFeaturesDict[key] = mddUnseenFeaturesDict[key].reshape(mddUnseenFeaturesDict[key].shape[0], -1)

    # Concatenate features across types
    hcFeature = np.concatenate([hcFeaturesDict[key] for key in hcFeaturesDict.keys()], axis=1)
    mddFeature = np.concatenate([mddFeaturesDict[key] for key in mddFeaturesDict.keys()], axis=1)
    hcFeatureUnseen = np.concatenate([hcUnseenFeaturesDict[key] for key in hcUnseenFeaturesDict.keys()], axis=1)
    mddFeatureUnseen = np.concatenate([mddUnseenFeaturesDict[key] for key in mddUnseenFeaturesDict.keys()], axis=1)

    return hcFeature, mddFeature, hcFeatureUnseen, mddFeatureUnseen

In [78]:
def get_model_config_dict(isHyperparamTuning, epochLength, featuresType, subFolderForSaveModel, channelsSelectedSetLst, channelsSelectedSetName):
    modma10sFeatPath = os.getenv("MODMA_10S_FEATURES_DIR")
    predct10sFeatPath = os.getenv("PREDCT_10S_FEATURES_DIR")

    modma15sFeatPath = os.getenv("MODMA_15S_FEATURES_DIR")
    predct15sFeatPath = os.getenv("PREDCT_15S_FEATURES_DIR")

    modma20sFeatPath = os.getenv("MODMA_20S_FEATURES_DIR")
    predct20sFeatPath = os.getenv("PREDCT_20S_FEATURES_DIR")

    modma30sFeatPath = os.getenv("MODMA_30S_FEATURES_DIR")
    predct30sFeatPath = os.getenv("PREDCT_30S_FEATURES_DIR")

    modma40sFeatPath = os.getenv("MODMA_40S_FEATURES_DIR")
    predct40sFeatPath = os.getenv("PREDCT_40S_FEATURES_DIR")

    modma45sFeatPath = os.getenv("MODMA_45S_FEATURES_DIR")
    predct45sFeatPath = os.getenv("PREDCT_45S_FEATURES_DIR")

    modma60sFeatPath = os.getenv("MODMA_60S_FEATURES_DIR")
    predct60sFeatPath = os.getenv("PREDCT_60S_FEATURES_DIR")

    if epochLength == 10:
        modmaFeatPath = modma10sFeatPath
        predctFeatPath = predct10sFeatPath
    elif epochLength == 15:
        modmaFeatPath = modma15sFeatPath
        predctFeatPath = predct15sFeatPath
    elif epochLength == 20:
        modmaFeatPath = modma20sFeatPath
        predctFeatPath = predct20sFeatPath
    elif epochLength == 30:
        modmaFeatPath = modma30sFeatPath
        predctFeatPath = predct30sFeatPath
    elif epochLength == 40:
        modmaFeatPath = modma40sFeatPath
        predctFeatPath = predct40sFeatPath
    elif epochLength == 45:
        modmaFeatPath = modma45sFeatPath
        predctFeatPath = predct45sFeatPath
    elif epochLength == 60:
        modmaFeatPath = modma60sFeatPath
        predctFeatPath = predct60sFeatPath
    
    pathForSaveModel = os.path.join(MODEL_MAIN_DIR_PATH, channelsSelectedSetName, subFolderForSaveModel)
    whichFeatureType = featuresType
    
    hcFeature, mddFeature, hcFeatureUnseen, mddFeatureUnseen = get_feature(whichFeatureType, modmaFeatPath, predctFeatPath, channelsSelectedSetLst)
    
    modelDict = {
        "is_hyperparam_tuning": isHyperparamTuning,
        "epoch_length": epochLength,
        "features_type": featuresType,
        "modma_features_dir_path": modmaFeatPath,
        "predct_features_dir_path": predctFeatPath,
        "save_model_dir_path": pathForSaveModel,
        "hc_feature": hcFeature,
        "mdd_feature": mddFeature,
        "hc_feature_unseen": hcFeatureUnseen,
        "mdd_feature_unseen": mddFeatureUnseen,
    }
    return modelDict

In [79]:
channelsSelectedSetDict = {
    "12chConfig_1": [ChannelsIndexMapping.Fp1.value, ChannelsIndexMapping.Fp2.value, ChannelsIndexMapping.F3.value,
                     ChannelsIndexMapping.F4.value, ChannelsIndexMapping.F7.value, ChannelsIndexMapping.F8.value,
                     ChannelsIndexMapping.P3.value, ChannelsIndexMapping.P4.value, ChannelsIndexMapping.P7.value,
                     ChannelsIndexMapping.P8.value, ChannelsIndexMapping.O1.value, ChannelsIndexMapping.O2.value],
    
    "12chConfig_2": [ChannelsIndexMapping.Fp1.value, ChannelsIndexMapping.Fp2.value, ChannelsIndexMapping.F3.value,
                     ChannelsIndexMapping.F4.value, ChannelsIndexMapping.F7.value, ChannelsIndexMapping.F8.value,
                     ChannelsIndexMapping.T7.value, ChannelsIndexMapping.T8.value, ChannelsIndexMapping.P7.value,
                     ChannelsIndexMapping.P8.value, ChannelsIndexMapping.O1.value, ChannelsIndexMapping.O2.value],
    
    "8chConfig_1": [ChannelsIndexMapping.Fp1.value, ChannelsIndexMapping.Fp2.value, ChannelsIndexMapping.F7.value,
                    ChannelsIndexMapping.F8.value, ChannelsIndexMapping.P7.value, ChannelsIndexMapping.P8.value,
                    ChannelsIndexMapping.O1.value, ChannelsIndexMapping.O2.value],
    
    "8chConfig_2": [ChannelsIndexMapping.Fp1.value, ChannelsIndexMapping.Fp2.value, ChannelsIndexMapping.T7.value,
                    ChannelsIndexMapping.T8.value, ChannelsIndexMapping.P7.value, ChannelsIndexMapping.P8.value,
                    ChannelsIndexMapping.O1.value, ChannelsIndexMapping.O2.value],
    
    "4chConfig_1": [ChannelsIndexMapping.Fp1.value, ChannelsIndexMapping.Fp2.value,
                    ChannelsIndexMapping.O1.value, ChannelsIndexMapping.O2.value],
    
    "4chConfig_2": [ChannelsIndexMapping.Fp1.value, ChannelsIndexMapping.Fp2.value,
                    ChannelsIndexMapping.F7.value, ChannelsIndexMapping.F8.value],
    
    "4chConfig_3": [ChannelsIndexMapping.P7.value, ChannelsIndexMapping.P8.value,
                    ChannelsIndexMapping.O1.value, ChannelsIndexMapping.O2.value],
    
    "4chConfig_4": [ChannelsIndexMapping.P3.value, ChannelsIndexMapping.P4.value,
                    ChannelsIndexMapping.O1.value, ChannelsIndexMapping.O2.value],
    
    "4chConfig_5": [ChannelsIndexMapping.Fp1.value, ChannelsIndexMapping.Fp2.value,
                    ChannelsIndexMapping.F3.value, ChannelsIndexMapping.F4.value]
}

In [80]:
configModelPipelineDict = {
    "10s_hfd_coherence": ["hfd", "coherence"],
    "15s_hfd_coherence": ["bandpower"],
    "15s_subbandpower": ["subbandpower"],
    "15s_bandpower_hfd_relativepower_subbandpower": ["bandpower", "hfd", "relativepower", "subbandpower"],
    "15s_bandpower_subbandpower_hfd": ["bandpower", "subbandpower", "hfd"],
    "30s_bandpower": ["bandpower"],
    "30s_bandpower_coherence_hfd_relativepower_subbandpower": ["bandpower", "coherence", "hfd", "relativepower", "subbandpower"],
    "30s_bandpower_subbandpower_hfd": ["bandpower", "subbandpower", "hfd"],
    "40s_bandpower_coherence_subbandpower": ["bandpower", "coherence", "subbandpower"],
    "60s_bandpower": ["bandpower"],
    "60s_subbandpower": ["subbandpower"],
    "60s_bandpower_subbandpower_hfd": ["bandpower", "subbandpower", "hfd"]
}

In [81]:
modelsConfigLst = []

for channelSetName, channelSetLst in channelsSelectedSetDict.items():
    for pipelineName, featureTypes in configModelPipelineDict.items():
        epochLength = int(pipelineName.split("_")[0][:-1])
        print(f"Epoch Length: {epochLength}, Pipeline Name: {pipelineName}, Feature Types: {featureTypes}")
        subFolderForSaveModel = f"20250406_{epochLength}s_{pipelineName}"
        
        configModel = get_model_config_dict(
            isHyperparamTuning=IS_HYPERPARAMETER_TUNING,
            epochLength=epochLength,
            featuresType=featureTypes,
            subFolderForSaveModel=subFolderForSaveModel,
            channelsSelectedSetLst=channelSetLst,
            channelsSelectedSetName=channelSetName
        )
        
        modelsConfigLst.append(configModel)

Epoch Length: 10, Pipeline Name: 10s_hfd_coherence, Feature Types: ['hfd', 'coherence']
Epoch Length: 15, Pipeline Name: 15s_hfd_coherence, Feature Types: ['bandpower']
Epoch Length: 15, Pipeline Name: 15s_subbandpower, Feature Types: ['subbandpower']
Epoch Length: 15, Pipeline Name: 15s_bandpower_hfd_relativepower_subbandpower, Feature Types: ['bandpower', 'hfd', 'relativepower', 'subbandpower']
Epoch Length: 15, Pipeline Name: 15s_bandpower_subbandpower_hfd, Feature Types: ['bandpower', 'subbandpower', 'hfd']
Epoch Length: 30, Pipeline Name: 30s_bandpower, Feature Types: ['bandpower']
Epoch Length: 30, Pipeline Name: 30s_bandpower_coherence_hfd_relativepower_subbandpower, Feature Types: ['bandpower', 'coherence', 'hfd', 'relativepower', 'subbandpower']
Epoch Length: 30, Pipeline Name: 30s_bandpower_subbandpower_hfd, Feature Types: ['bandpower', 'subbandpower', 'hfd']
Epoch Length: 40, Pipeline Name: 40s_bandpower_coherence_subbandpower, Feature Types: ['bandpower', 'coherence', 'subb

In [ ]:
models = {
    "KNN": (KNeighborsClassifier(), {"n_neighbors": [3, 5, 7, 9], "weights": ["uniform", "distance"]}),
    "SVM": (SVC(probability=True, random_state=42), {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}),
    "Decision Tree": (DecisionTreeClassifier(), {"max_depth": [5, 10, 15]}),
    "Random Forest": (RandomForestClassifier(), {"n_estimators": [50, 100], "max_depth": [10, 20]}),
    "Logistic Regression": (LogisticRegression(), {"C": [0.01, 0.1, 1]}),
    "Gradient Boosting": (GradientBoostingClassifier(), {"n_estimators": [50, 100], "learning_rate": [0.01, 0.1, 0.2]}),
    "AdaBoost": (AdaBoostClassifier(), {"n_estimators": [50, 100], "learning_rate": [0.01, 0.1, 1]}),
}

featureSelectors = {
    "NoFeatureSelection": None,
    "SelectKBest": SelectKBest(score_func=f_classif, k=100)
}

def get_stratified_kfold_data(X, y, nSplits=5):
    skf = StratifiedKFold(n_splits=nSplits, shuffle=True, random_state=42)
    return skf.split(X, y)

In [ ]:
results = []

for modelConfig in modelsConfigLst:  # Loop through each model configuration in the list
    print(f"Training model for features: {modelConfig['features_type']}")

    # Prepare training and testing data
    hcFeatures = modelConfig["hc_feature"]
    mddFeatures = modelConfig["mdd_feature"]
    hcUnseenFeatures = modelConfig["hc_feature_unseen"]
    mddUnseenFeatures = modelConfig["mdd_feature_unseen"]

    X_train = np.concatenate((hcFeatures, mddFeatures), axis=0)
    y_train = np.concatenate((np.zeros(hcFeatures.shape[0]), np.ones(mddFeatures.shape[0])), axis=0)

    X_test = np.concatenate((hcUnseenFeatures, mddUnseenFeatures), axis=0)
    y_test = np.concatenate((np.zeros(hcUnseenFeatures.shape[0]), np.ones(mddUnseenFeatures.shape[0])), axis=0)

    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")

    num_features = X_train.shape[1]  # Get the number of features (second dimension)

    for modelName, (model, paramGrid) in models.items():
        for featureSelectorName, featureSelector in featureSelectors.items():
            print(f"Training {modelName} with {featureSelectorName}...")
            if featureSelector is not None:
                selector = featureSelector
            else:
                selector = None
            skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            bestFold = None
            bestAcc = -np.inf
            bestFoldModel = None
            bestFoldIdx = None
            hyperparamTuningDetails = []

            # Hyperparameter tuning
            if modelConfig["is_hyperparam_tuning"] and paramGrid:
                gridSearch = GridSearchCV(model, paramGrid, cv=10, scoring="accuracy", n_jobs=-1, return_train_score=True)
                gridSearch.fit(X_train, y_train)
                model = gridSearch.best_estimator_
                bestParams = gridSearch.best_params_
                print(f"Best hyperparameters for {modelName}: {bestParams}")

                for i, params in enumerate(gridSearch.cv_results_["params"]):
                    meanTestScore = gridSearch.cv_results_["mean_test_score"][i]
                    stdTestScore = gridSearch.cv_results_["std_test_score"][i]
                    hyperparamTuningDetails.append({
                        "params": params,
                        "mean_test_score": meanTestScore,
                        "std_test_score": stdTestScore
                    })
            else:
                bestParams = None

            foldAccuracies = []
            foldPrecisions = []
            foldRecalls = []
            foldF1Scores = []
            selectedFeatures = []
            foldHcTrain = []
            foldMddTrain = []
            foldHcVal = []
            foldMddVal = []
            foldHcTest = []
            foldMddTest = []

            for fold, (trainIdx, valIdx) in enumerate(skf.split(X_train, y_train)):
                XTrain, XVal = X_train[trainIdx], X_train[valIdx]
                yTrain, yVal = y_train[trainIdx], y_train[valIdx]

                hcTrain = int(np.sum(yTrain == 0))
                mddTrain = int(np.sum(yTrain == 1))
                hcVal = int(np.sum(yVal == 0))
                mddVal = int(np.sum(yVal == 1))
                hcTest = int(np.sum(y_test == 0))
                mddTest = int(np.sum(y_test == 1))

                foldHcTrain.append(hcTrain)
                foldMddTrain.append(mddTrain)
                foldHcVal.append(hcVal)
                foldMddVal.append(mddVal)
                foldHcTest.append(hcTest)
                foldMddTest.append(mddTest)

                if selector is not None:
                    XTrain = selector.fit_transform(XTrain, yTrain)
                    XVal = XVal[:, selector.get_support()]

                model.fit(XTrain, yTrain)

                yPred = model.predict(XVal)
                acc = accuracy_score(yVal, yPred)
                prec = precision_score(yVal, yPred)
                rec = recall_score(yVal, yPred)
                f1 = f1_score(yVal, yPred)

                foldAccuracies.append(float(acc))
                foldPrecisions.append(float(prec))
                foldRecalls.append(float(rec))
                foldF1Scores.append(float(f1))

                if selector is not None:
                    selectedFeatures.append(np.where(selector.get_support())[0].tolist())

                modelDir = os.path.join(modelConfig["save_model_dir_path"], f"{modelName}_{featureSelectorName}")
                if not os.path.exists(modelDir):
                    os.makedirs(modelDir)
                modelFilename = os.path.join(modelDir, f"model_fold_{fold + 1}.pkl")
                joblib.dump(model, modelFilename)

                if acc > bestAcc:
                    bestAcc = acc
                    bestFold = model
                    bestFoldIdx = fold

            print(f"Best fold: {bestFoldIdx + 1} with ACC score: {bestAcc:.4f}")

            if selector is not None:
                X_train_selected = selector.fit_transform(X_train, y_train)
                X_test_selected = X_test[:, selector.get_support()]
            else:
                X_train_selected = X_train
                X_test_selected = X_test

            bestFold.fit(X_train_selected, y_train)
            yTestPred = bestFold.predict(X_test_selected)
            finalAccuracy = accuracy_score(y_test, yTestPred)
            finalPrecision = precision_score(y_test, yTestPred)
            finalRecall = recall_score(y_test, yTestPred)
            finalF1 = f1_score(y_test, yTestPred)

            yProb = bestFold.predict_proba(X_test_selected)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, yProb)
            roc_auc = auc(fpr, tpr)
            fig, ax = plt.subplots(1, 2, figsize=(12, 5))
            ax[0].plot(fpr, tpr, color='blue', label=f'ROC curve (area = {roc_auc:.2f}')
            ax[0].plot([0, 1], [0, 1], color='gray', linestyle='--')
            ax[0].set_xlabel('False Positive Rate')
            ax[0].set_ylabel('True Positive Rate')
            ax[0].set_title('ROC Curve')
            ax[0].legend(loc='lower right')

            cm = confusion_matrix(y_test, yTestPred)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
            ax[1].set_xlabel('Predicted Label')
            ax[1].set_ylabel('True Label')
            ax[1].set_title('Confusion Matrix')

            plotFilename = os.path.join(modelConfig["save_model_dir_path"], f"{modelName}_{featureSelectorName}_best_fold_roc_cm.png")
            plt.savefig(plotFilename)

            results.append({
                "model": modelName,
                "feature_selection": featureSelectorName,
                "features_type": modelConfig["features_type"],
                "num_features": num_features,
                "avg_accuracy": f"{np.mean(foldAccuracies):.4f} ± {np.std(foldAccuracies):.4f}",
                "avg_precision": f"{np.mean(foldPrecisions):.4f} ± {np.std(foldPrecisions):.4f}",
                "avg_recall": f"{np.mean(foldRecalls):.4f} ± {np.std(foldRecalls):.4f}",
                "avg_f1_score": f"{np.mean(foldF1Scores):.4f} ± {np.std(foldF1Scores):.4f}",
                "final_accuracy": f"{finalAccuracy:.4f}",
                "final_precision": f"{finalPrecision:.4f}",
                "final_recall": f"{finalRecall:.4f}",
                "final_f1_score": f"{finalF1:.4f}",
                "significant_features": np.unique([item for sublist in selectedFeatures for item in sublist]).tolist(),
                "folds_accuracy": foldAccuracies,
                "folds_precision": foldPrecisions,
                "folds_recall": foldRecalls,
                "folds_f1_score": foldF1Scores,
                "fold_hc_train": [int(x) for x in foldHcTrain], 
                "fold_mdd_train": [int(x) for x in foldMddTrain],  
                "fold_hc_val": [int(x) for x in foldHcVal], 
                "fold_mdd_val": [int(x) for x in foldMddVal],  
                "fold_hc_test": [int(x) for x in foldHcTest],  
                "fold_mdd_test": [int(x) for x in foldMddTest],
                "best_fold": bestFoldIdx + 1,
                "best_hyperparameters": bestParams,
                "hyperparam_tuning_details": hyperparamTuningDetails
            })

    # Save results for this model configuration
    saveDir = modelConfig["save_model_dir_path"]
    resultsDf = pd.DataFrame(results)
    csvFilename = os.path.join(saveDir, f"results_{num_features}_features.csv")
    excelFilename = os.path.join(saveDir, f"results_{num_features}_features.xlsx")
    resultsDf.to_csv(csvFilename, index=False)
    resultsDf.to_excel(excelFilename, index=False)
    print(f"Results saved to {csvFilename} and {excelFilename}")

# Print results
for result in results:
    print(f"\n{result['model']} with {result['feature_selection']} - Final evaluation")
    print(f"Final Accuracy: {result['final_accuracy']}")
    print(f"Final Precision: {result['final_precision']}")
    print(f"Final Recall: {result['final_recall']}")
    print(f"Final F1 Score: {result['final_f1_score']}")
    if result["best_hyperparameters"]:
        print(f"Best Hyperparameters: {result['best_hyperparameters']}")